In [206]:
import os
import ROOT as r
import uproot
import time
import argparse
import pandas as pd
from datetime import datetime
import numpy as np
import json
from IPython.display import display
import awkward as ak

In [207]:
class runInfo:
    
    def __init__(self):
        
        self.run = -1
        self.file = -1
        self.rawDirectory = None
        self.offlineDirectory = None
        self.daqFile = None
        self.trigFile = None
        self.matchFile = None
        self.offlineFile = None
        self.totalEvents = 0
        self.unmatchedEvents = 0
        self.startTime = -1
        self.unmatchedBoards = 0
        self.daqCTime = None
        self.trigCTime = None
        self.matchCTime = None
        self.offlineCTime = None
        self.offlineTrigMatched = 0
        self.triggerConfigPassing = False
        self.activeChannels = np.full(80, False)
    
    def __str__(self):
        out = 'run: {0} file: {1}\n\t\
               daqFile: {2}\n\t\
               trigFile: {3}\n\t\
               matchFile: {4}\n\t\
               offlineFile: {5}'.format(self.run, self.file, self.daqFile, self.trigFile, self.matchFile, self.offlineFile)
        return out
        
        
    

In [208]:
rawDirectories = ['1300']
#rawSubDirectories = ['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009']
rawSubDirectories = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009']

usedChannels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                21, 22, 23, 26, 27, 28, 29, 30, 
                31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
                41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
                51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 
                61, 62, 63, 68, 69, 70, 
                71, 72, 73, 74, 75, 78, 79]

In [238]:
class fileChecker():
    
    def __init__(self):
        self.min_run = 1200
        self.max_run = 1300
        self.rawDir = ''
        self.offlineDir = ''        
        
        self.parse_args()
        if self.args.rawDir: self.rawDir = self.args.rawDir
        if self.args.offlineDir: self.offlineDir = self.args.offlineDir
            
        self.initializePlots()

        self.daqFiles = {}
        self.trigFiles = {}
        self.matchedFiles = {}
        self.offlineFiles = {}
        
        self.runInfos = pd.DataFrame(columns=['run', 'file', 'rawDir', 'offlineDir', 'daqFile', 'trigFile',
                                  'matchFile', 'offlineFile', 'totalEvents', 'unmatchedEvents', 
                                  'startTime', 'unmatchedBoards', 'daqCTime', 'trigCTime', 'matchCTime',
                                  'offlineCTime', 'offlineTrigMatched', 'triggerConfigPassing', 'activeChannels'])
        
        self.debug = False
        
        self.rawDirs = rawDirectories = ['1000', '1100', '1200']
        self.subRawDirs = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009']
        
    def parse_args(self):
        parser=argparse.ArgumentParser()
        parser.add_argument("-r", "--rawDir", type=str, default = '/store/user/milliqan/run3/bar/', help="Raw data directory")
        parser.add_argument("-o", "--offlineDir", type=str, default = '/store/user/milliqan/trees/v34/', help="Offline data directory")
        self.args = parser.parse_args(args=[])

    def initializePlots(self):

        bins = self.max_run - self.min_run
        self.h_total = r.TH1F("h_total", "Total Number of Events in Run", bins, self.min_run, self.max_run)
        self.h_unmatched = r.TH1F("h_unmatched", "Number of Unmatched Events in Run", bins, self.min_run, self.max_run)
        self.h_startTimes = r.TH1F("h_startTimes", "Start Times of Runs", bins, self.min_run, self.max_run)
        self.h_boardUnmatched = r.TH1F("h_boardUnmatched", "Number of Boards Unmatched", bins, self.min_run, self.max_run)

        self.c1  = r.TCanvas("c1", "c1", 800,800)

    def checkActiveTriggers(self, trigger):
        t_bin = trigger & 0b0000000001011111 #check only first 8 triggers
        minimumTriggers = t_bin & 0b01011111 #(1011111)
        if minimumTriggers == 95: 
            print("Trigger passing", minimumTriggers)
            return True
        else: 
            print("Trigger failing", minimumTriggers)
            return False
        
    def checkOfflineFiles(self, fileList):

        total_unmatched = 0
        for events in uproot.iterate(

            #files
            fileList,

            #branches
            ['runNumber', 'fileNumber', 'boardsMatched', 'tStartTime', 'tTrigger'],

            #cut
            #cut="",

            how="zip",

            step_size=10000,

            num_workers=8,

            ):

            unmatchedCut = events[:, "boardsMatched"] == 0
            unmatched = events[unmatchedCut, "boardsMatched"]
            self.h_boardUnmatched.Fill(events[0, 'runNumber'], len(unmatched))

            unmatchedEventCut = events[:, "tTrigger"] != -1
            unmatchedEvents = events[unmatchedEventCut, "tTrigger"]
            
            self.runInfos['unmatchedBoards'].loc[(self.runInfos['run'] == events[0, 'runNumber']) & (self.runInfos['file'] == events[0, 'fileNumber'])] += len(unmatched)
            self.runInfos['offlineTrigMatched'].loc[(self.runInfos['run'] == events[0, 'runNumber']) & (self.runInfos['file'] == events[0, 'fileNumber'])] += len(unmatchedEvents)
        
        for events in uproot.iterate(

            #files
            fileList,

            #branches
            ['runNumber', 'fileNumber', 'chan'],

            step_size=10000,

            num_workers=8,

            ):

            for i in range(80):
                chanPulses = ak.any(events[:, 'chan'] == i)
                self.runInfos.at[self.runInfos.index[(self.runInfos['run'] == events[0, 'runNumber']) & (self.runInfos['file'] == events[0, 'fileNumber'])][0], 'activeChannels'][i] = chanPulses

    
    def getRunFile(self, filename):
        runNum = filename.split('Run')[-1].split('.')[0]
        fileNum = filename.split('.')[1].split('_')[0]
        return runNum, fileNum
    
    
    def saveJson(self, name):
        self.runInfos.to_json(name, orient = 'split', compression = 'infer', index = 'true')
        
    def loadJson(self, jsonFile):
        fin = open(jsonFile)
        data = json.load(fin)
        self.runInfos = pd.DataFrame(data['data'], columns=data['columns'])
    
    def checkMatchedFiles(self, fileList):
        total_unmatched = 0
        for events in uproot.iterate(

            #files
            fileList,

            #branches
            ['runNum', 'eventNum', 'trigger', 'startTime'],


            #needs to be 1000 events for file number to be correct
            step_size=10000,

            num_workers=8,
            
        ):

            unmatchedCut = events[:, "trigger"] == -1
            unmatched = events[unmatchedCut, "trigger"]
            self.h_unmatched.Fill(events[0, 'runNum'], len(unmatched))
            self.h_total.Fill(events[0, 'runNum'], len(events))
            thisbin = self.h_startTimes.FindBin(events[0, 'runNum'])
            self.h_startTimes.SetBinContent(thisbin, events[0, 'startTime'])
            
            fileNum = events[0, 'eventNum']/1000 + 1
            
            #ToDo sometimes the first event has no start time, need to check a few other events 
            #self.runInfos['startTime'].loc[(self.runInfos['run'] == events[0, 'runNum']) & (self.runInfos['file'] == fileNum)] = events[0, 'startTime']
            
            self.runInfos['totalEvents'].loc[(self.runInfos['run'] == events[0, 'runNum']) & (self.runInfos['file'] == fileNum)] += len(events)
            self.runInfos['unmatchedEvents'].loc[(self.runInfos['run'] == events[0, 'runNum']) & (self.runInfos['file'] == fileNum)] += len(unmatched)
    
        #TODO temporary hack to get trigger board meta data, eventually move these checks to offline checks
        fileListMeta = [x.split(':')[0] for x in fileList]

        for file in fileListMeta:
            myfile = r.TFile.Open(file, 'read')
            metadata = myfile.Get("MetaData")
            passing = True
            runNum, fileNum = self.getRunFile(file)
            for events in metadata:
                passing = passing and events.fwVersion >= 11
                passing = passing and self.checkActiveTriggers(events.trigger)
                passing = passing and events.coincidenceTime == 10
                passing = passing and events.deadTime == 142
            #print("Trigger board file {} meta data passing {}, run {}, file {}".format(file, passing, runNum, fileNum))
            self.runInfos.loc[(self.runInfos['run'] == int(runNum)) & (self.runInfos['file'] == int(fileNum)), 'triggerConfigPassing'] = passing
    
    def getOfflineInfo(self):
        #add loc statement to only get those that haven't been updated yet
        rawFiles = self.runInfos[['run', 'file']].loc[~pd.notnull(self.runInfos['offlineFile'])].to_numpy()
        for pair in rawFiles:
            offlineFile = 'MilliQan_Run{0}.{1}_v34.root'.format(pair[0], pair[1])
            if os.path.exists(self.offlineDir+'/'+offlineFile): 
                self.runInfos['offlineFile'].loc[(self.runInfos['run'] == pair[0]) & (self.runInfos['file'] == pair[1])] = offlineFile
                self.runInfos['offlineDir'].loc[(self.runInfos['run'] == pair[0]) & (self.runInfos['file'] == pair[1])] = self.offlineDir
                self.runInfos['offlineCTime'].loc[(self.runInfos['run'] == pair[0]) & (self.runInfos['file'] == pair[1])] = datetime.fromtimestamp(os.path.getctime(self.offlineDir+'/'+offlineFile))
            else:
                print("File {0} does not exist".format(self.offlineDir+'/'+offlineFile))

    def getRawInfo(self, directory):
        fullPath = self.rawDir+directory
        if not os.path.isdir(fullPath): 
            print("Directory {0} does not exist, skipping...".format(fullPath))
            return
        for ifile, filename in enumerate(os.listdir(fullPath)):
            if not filename.endswith('.root'): continue
            if not filename.startswith('MilliQan'): continue
            if self.debug and len(self.runInfos) > 10: break

            thisRun = runInfo()
            runNum, fileNum = self.getRunFile(filename)
            thisRun.run = int(runNum)
            thisRun.file = int(fileNum)
            thisRun.daqFile = filename
            thisRun.rawDir = fullPath
            thisRun.daqCTime = datetime.fromtimestamp(os.path.getctime(fullPath+'/'+filename))
            trigName = "TriggerBoard_Run{0}.{1}.root".format(runNum, fileNum)
            matchName = "MatchedEvents_Run{0}.{1}_rematch.root".format(runNum, fileNum)
            if os.path.exists(fullPath+'/'+trigName): 
                thisRun.trigFile = trigName
                thisRun.trigCTime = datetime.fromtimestamp(os.path.getctime(fullPath+'/'+trigName))
            if os.path.exists(fullPath+'/'+matchName): 
                thisRun.matchFile = matchName
                thisRun.matchCTime = datetime.fromtimestamp(os.path.getctime(fullPath+'/'+matchName))

            self.runInfos.loc[len(self.runInfos.index)] = thisRun.__dict__
                            
    def runCheckMatchedFiles(self):
        runs = self.runInfos.run.unique()
        for run in runs:
            runList = self.runInfos[['rawDir', 'matchFile']].loc[(self.runInfos['run']==run) & (self.runInfos['totalEvents']==0)].apply(lambda x: '/'.join((x.rawDir, x.matchFile)) if (x.rawDir!=None and x.matchFile!=None) else None, axis=1).values.tolist()
            #print(runList)
            runList = [x+':matchedTrigEvents' for x in runList if x!=None]
            if len(runList) == 0: continue
            self.checkMatchedFiles(runList)
        
    def runCheckOfflineFiles(self):
        #now look at offline files post processing info
        runs = self.runInfos.run.unique()
        for run in runs:
            print("Run:", run)
            runList = self.runInfos[['offlineDir', 'offlineFile']].loc[(self.runInfos['run']==run) & (self.runInfos['unmatchedBoards']==0)].apply(lambda x: '/'.join((x.offlineDir, x.offlineFile)) if (x.offlineDir!=None and x.offlineFile!=None) else None, axis=1).values.tolist()
            runList = [x+':t' for x in runList if x!=None]
            if len(runList) == 0: continue
            self.checkOfflineFiles(runList)
    
    #function to do all tasks once per directory and update json
    def looper(self, dirs=[], subDirs=[], jsonName='checkMatching.json'):
        if len(dirs)==0:
            dirs = self.rawDirs
        if len(subDirs)==0:
            subDirs = self.subRawDirs
        for d1 in dirs:
            for d2 in subDirs:
                print("Running over directory {0}/{1}".format(d1, d2))
                self.getRawInfo('{0}/{1}'.format(d1, d2))
                self.getOfflineInfo()
                self.runCheckMatchedFiles()
                self.runCheckOfflineFiles()
                self.finalChecks()
                self.saveJson(jsonName)

    def finalChecks(self):

        self.runInfos['TriggersMatched'] = self.runInfos['unmatchedEvents'].apply(lambda x: True if x < 10 else False)
        self.runInfos['OfflineFilesTrigMatched'] = self.runInfos.apply(lambda x: True if (x['totalEvents'] > 0 and (x['offlineTrigMatched'] / x['totalEvents']) > 0.99) else False, axis=1)
        self.runInfos['passBoardMatching'] = self.runInfos.apply(lambda x: True if (x['totalEvents'] > 0 and (x['unmatchedBoards'] / x['totalEvents']) < 0.01) else False, axis=1)
        self.runInfos['passActiveChannels'] = self.runInfos.apply(lambda x: False if np.any(x['activeChannels'][usedChannels]==False) else True, axis=1)
        self.runInfos['inactiveChannels'] = self.runInfos.apply(lambda x: np.intersect1d(np.where(x['activeChannels']==False), usedChannels), axis=1)

        self.runInfos['goodRuns'] = self.runInfos['TriggersMatched'] & self.runInfos['OfflineFilesTrigMatched'] & \
                                    self.runInfos['passBoardMatching'] & self.runInfos['triggerConfigPassing'] #& \
                                    #self.runInfos['passActiveChannels'] #remove condition for now to create era appropriate missing channels

    def makeGoodRunList(self):
        
        goodRunList = pd.read_json("goodRunList.json", orient='split')
        
        goodRuns = self.runInfos
        print("Rows initial", goodRuns.shape[0])
        
        #criteria for good runs
        goodRuns = goodRuns.drop(goodRuns.loc[goodRuns['goodRuns']==False].index)
        print("Rows Final", goodRuns.shape[0])
        
        goodRuns = goodRuns[['run', 'file']]
        
        goodRunList = pd.concat([goodRunList, goodRuns])
        goodRunList.drop_duplicates()
        
        goodRunList.to_json("goodRunList.json", orient = 'split', compression = 'infer', index = 'true')
        
                
    def customStyle(self, s):
        colors = ['background-color: white']*len(s)
        if s.unmatchedEvents > 10:
            color = 'yellow'
            if s.unmatchedEvents > 100: color = 'red'
            colors[s.keys().get_loc('unmatchedEvents')] = 'background-color: %s' % color
        if s.unmatchedBoards > 5:
            color = 'yellow'
            if s.unmatchedBoards > 50: color = 'red'
            colors[s.keys().get_loc('unmatchedBoards')] = 'background-color: %s' % color
        if s.startTime == -1:
            colors[s.keys().get_loc('startTime')] = 'background-color: red'    
        if s.trigFile == '':
            colors[s.keys().get_loc('trigFile')] = 'background-color: red'
        if s.matchFile == '':
            colors[s.keys().get_loc('matchFile')] = 'background-color: red'
        if s.offlineFile == '':
            colors[s.keys().get_loc('offlineFile')] = 'background-color: red'

        return colors

    def printInfo(self):
        display(self.runInfos.style.apply(self.customStyle, axis=1))
        


In [240]:
if __name__ == "__main__":
    
    myfileChecker = fileChecker()
    myfileChecker.debug = False
    
    #myfileChecker.looper(dirs=['1100'], subDirs=['0007', '0008', '0009'], jsonName='checkNew.json')
    myfileChecker.looper(dirs=['1100'], subDirs=['0001'], jsonName='check1400_debug.json')

    myfileChecker.printInfo()
    
    myfileChecker.makeGoodRunList()


Running over directory 1100/0001
File /store/user/milliqan/trees/v34//MilliQan_Run1110.1_v34.root does not exist


/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-pandas/1.2.2-2b839ca39b0133bca5b0762d2e395686/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Trigger failing 0
Trigger failing 0
Trigger failing 0
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing 95
Trigger passing

In [124]:
def customStyle(s):
    colors = ['background-color: white']*len(s)
    if s.unmatchedEvents > 10:
        colors[s.keys().get_loc('unmatchedEvents')] = 'background-color: red'
    if s.unmatchedBoards > 5:
        colors[s.keys().get_loc('unmatchedBoards')] = 'background-color: red'
    if s.startTime == -1:
        colors[s.keys().get_loc('startTime')] = 'background-color: red'    
    if s.trigFile == None or s.trigFile == '':
        colors[s.keys().get_loc('trigFile')] = 'background-color: red'
    if s.matchFile == None:
        colors[s.keys().get_loc('matchFile')] = 'background-color: red'
    if s.offlineFile == None:
        colors[s.keys().get_loc('offlineFile')] = 'background-color: red'

    return colors

myfileChecker.runInfos.style.apply(customStyle, axis=1)

,run,file,rawDir,offlineDir,daqFile,trigFile,matchFile,offlineFile,totalEvents,unmatchedEvents,startTime,unmatchedBoards,daqCTime,trigCTime,matchCTime,offlineCTime,offlineTrigMatched,triggerConfigPassing,TriggersMatched,OfflineFilesTrigMatched,passBoardMatching,goodRuns
0,1400,1,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1400.1_default.root,TriggerBoard_Run1400.1.root,MatchedEvents_Run1400.1_rematch.root,MilliQan_Run1400.1_v34.root,1000,1000,-1,1000,2024-01-08 16:47:17,2024-01-08 16:46:17,2024-02-08 05:09:10,2024-01-08 17:08:33,0,True,False,False,False,False
1,1400,2,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1400.2_default.root,TriggerBoard_Run1400.2.root,MatchedEvents_Run1400.2_rematch.root,MilliQan_Run1400.2_v34.root,1000,1000,-1,1000,2024-01-08 16:46:43,2024-01-08 16:45:33,2024-02-08 05:09:11,2024-01-08 17:08:29,0,True,False,False,False,False
2,1400,3,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1400.3_default.root,TriggerBoard_Run1400.3.root,MatchedEvents_Run1400.3_rematch.root,MilliQan_Run1400.3_v34.root,1000,1000,-1,1000,2024-01-08 16:46:30,2024-01-08 16:45:16,2024-02-08 05:09:10,2024-01-08 17:09:19,0,True,False,False,False,False
3,1400,4,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1400.4_default.root,None,MatchedEvents_Run1400.4_rematch.root,MilliQan_Run1400.4_v34.root,1000,1000,-1,1000,2024-01-08 16:46:12,None,2024-02-08 05:09:11,2024-01-08 17:09:18,0,True,False,False,False,False
4,1400,5,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1400.5_default.root,None,MatchedEvents_Run1400.5_rematch.root,MilliQan_Run1400.5_v34.root,1000,1000,-1,1000,2024-01-08 16:45:59,None,2024-02-08 05:09:10,2024-01-08 17:13:15,0,True,False,False,False,False
5,1400,6,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1400.6_default.root,None,MatchedEvents_Run1400.6_rematch.root,MilliQan_Run1400.6_v34.root,1000,1000,-1,1000,2024-01-08 16:45:47,None,2024-02-08 05:09:10,2024-01-08 17:13:33,0,True,False,False,False,False
6,1400,7,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1400.7_default.root,None,MatchedEvents_Run1400.7_rematch.root,MilliQan_Run1400.7_v34.root,730,730,-1,730,2024-01-08 16:45:27,None,2024-02-08 05:09:10,2024-01-08 17:12:37,0,True,False,False,False,False
7,1401,100,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1401.100_default.root,TriggerBoard_Run1401.100.root,MatchedEvents_Run1401.100_rematch.root,MilliQan_Run1401.100_v34.root,1000,999,-1,0,2024-01-09 00:07:05,2024-01-09 00:05:57,2024-02-08 05:43:05,2024-01-09 01:03:10,0,True,False,False,True,False
8,1401,101,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1401.101_default.root,TriggerBoard_Run1401.101.root,MatchedEvents_Run1401.101_rematch.root,MilliQan_Run1401.101_v34.root,1000,1000,-1,0,2024-01-09 00:05:51,2024-01-09 00:05:17,2024-02-08 05:43:05,2024-01-09 01:03:13,0,True,False,False,True,False
9,1401,102,/store/user/milliqan/run3/bar/1400/0000,/store/user/milliqan/trees/v34/,MilliQan_Run1401.102_default.root,TriggerBoard_Run1401.102.root,MatchedEvents_Run1401.102_rematch.root,MilliQan_Run1401.102_v34.root,1000,1000,-1,0,2024-01-09 00:05:12,2024-01-09 00:04:38,2024-02-08 05:43:02,2024-01-09 01:05:47,0,True,False,False,True,False


In [ ]:
if __name__ == "__main__":
    
    myfileChecker = fileChecker()
    myfileChecker.debug = True
    
    #myfileChecker.looper(dirs=['1100'], subDirs=['0007', '0008', '0009'], jsonName='checkNew.json')
    myfileChecker.looper(dirs=['1400'], subDirs=['0001'], jsonName='check1400_debug.json')

    myfileChecker.printInfo()
    
    myfileChecker.makeGoodRunList()


Running over directory 1400/0001


/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-pandas/1.2.2-2b839ca39b0133bca5b0762d2e395686/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,run,file,rawDir,offlineDir,daqFile,trigFile,matchFile,offlineFile,totalEvents,unmatchedEvents,startTime,unmatchedBoards,daqCTime,trigCTime,matchCTime,offlineCTime,offlineTrigMatched,triggerConfigPassing,activeChannels,TriggersMatched,OfflineFilesTrigMatched,passBoardMatching,passActiveChannels,goodRuns
0,1410,1,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1410.1_default.root,TriggerBoard_Run1410.1.root,MatchedEvents_Run1410.1_rematch.root,MilliQan_Run1410.1_v34.root,999,3,-1,0,2024-01-15 10:20:31,2024-01-15 10:20:35,2024-02-08 05:21:30,2024-01-31 04:24:40,996,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False True False False True True True True True True True True False False True True],True,True,True,True,True
1,1411,100,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.100_default.root,TriggerBoard_Run1411.100.root,MatchedEvents_Run1411.100_rematch.root,MilliQan_Run1411.100_v34.root,1000,0,-1,0,2024-01-16 00:22:55,2024-01-16 00:22:32,2024-02-08 05:52:48,2024-01-31 04:20:59,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True True True True False False True True],True,True,True,True,True
2,1411,101,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.101_default.root,TriggerBoard_Run1411.101.root,MatchedEvents_Run1411.101_rematch.root,MilliQan_Run1411.101_v34.root,1000,0,-1,0,2024-01-16 00:27:55,2024-01-16 00:27:33,2024-02-08 05:52:50,2024-01-31 04:27:40,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True True True True False False True True],True,True,True,True,True
3,1411,102,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.102_default.root,TriggerBoard_Run1411.102.root,MatchedEvents_Run1411.102_rematch.root,MilliQan_Run1411.102_v34.root,1000,0,-1,0,2024-01-16 00:37:56,2024-01-16 00:37:33,2024-02-08 05:52:49,2024-01-31 04:23:46,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True True True True False False True True],True,True,True,True,True
4,1411,103,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.103_default.root,TriggerBoard_Run1411.103.root,MatchedEvents_Run1411.103_rematch.root,MilliQan_Run1411.103_v34.root,1000,0,-1,0,2024-01-16 00:47:56,2024-01-16 00:47:33,2024-02-08 05:52:49,2024-01-31 04:24:44,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True T

Rows initial 11
Rows Final 11


Warning in <TROOT::Append>: Replacing existing TH1: h_total (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_unmatched (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_startTimes (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_boardUnmatched (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [ ]:
if __name__ == "__main__":
    
    myfileChecker = fileChecker()
    myfileChecker.debug = True
    
    #myfileChecker.looper(dirs=['1100'], subDirs=['0007', '0008', '0009'], jsonName='checkNew.json')
    myfileChecker.looper(dirs=['1400'], subDirs=['0001'], jsonName='check1400_debug.json')

    myfileChecker.printInfo()
    
    myfileChecker.makeGoodRunList()


Running over directory 1400/0001


/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-pandas/1.2.2-2b839ca39b0133bca5b0762d2e395686/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,run,file,rawDir,offlineDir,daqFile,trigFile,matchFile,offlineFile,totalEvents,unmatchedEvents,startTime,unmatchedBoards,daqCTime,trigCTime,matchCTime,offlineCTime,offlineTrigMatched,triggerConfigPassing,activeChannels,TriggersMatched,OfflineFilesTrigMatched,passBoardMatching,passActiveChannels,goodRuns
0,1410,1,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1410.1_default.root,TriggerBoard_Run1410.1.root,MatchedEvents_Run1410.1_rematch.root,MilliQan_Run1410.1_v34.root,999,3,-1,0,2024-01-15 10:20:31,2024-01-15 10:20:35,2024-02-08 05:21:30,2024-01-31 04:24:40,996,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False True False False True True True True True True True True False False True True],True,True,True,True,True
1,1411,100,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.100_default.root,TriggerBoard_Run1411.100.root,MatchedEvents_Run1411.100_rematch.root,MilliQan_Run1411.100_v34.root,1000,0,-1,0,2024-01-16 00:22:55,2024-01-16 00:22:32,2024-02-08 05:52:48,2024-01-31 04:20:59,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True True True True False False True True],True,True,True,True,True
2,1411,101,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.101_default.root,TriggerBoard_Run1411.101.root,MatchedEvents_Run1411.101_rematch.root,MilliQan_Run1411.101_v34.root,1000,0,-1,0,2024-01-16 00:27:55,2024-01-16 00:27:33,2024-02-08 05:52:50,2024-01-31 04:27:40,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True True True True False False True True],True,True,True,True,True
3,1411,102,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.102_default.root,TriggerBoard_Run1411.102.root,MatchedEvents_Run1411.102_rematch.root,MilliQan_Run1411.102_v34.root,1000,0,-1,0,2024-01-16 00:37:56,2024-01-16 00:37:33,2024-02-08 05:52:49,2024-01-31 04:23:46,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True True True True False False True True],True,True,True,True,True
4,1411,103,/store/user/milliqan/run3/bar/1400/0001,/store/user/milliqan/trees/v34/,MilliQan_Run1411.103_default.root,TriggerBoard_Run1411.103.root,MatchedEvents_Run1411.103_rematch.root,MilliQan_Run1411.103_v34.root,1000,0,-1,0,2024-01-16 00:47:56,2024-01-16 00:47:33,2024-02-08 05:52:49,2024-01-31 04:24:44,1000,True,[ True True True True True True True True True True True True True True True True True True True True True True True True False False True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True False False False False True True True True True T

Rows initial 11
Rows Final 11


Warning in <TROOT::Append>: Replacing existing TH1: h_total (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_unmatched (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_startTimes (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_boardUnmatched (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [ ]:
myfileChecker.runInfos

In [ ]:
newchecker = fileChecker()
newchecker.loadJson('check1300.json')
newchecker.printInfo()

In [ ]:
newchecker.runInfos.loc[(newchecker.runInfos['unmatchedEvents'] > 5)]

In [ ]:
import matplotlib.pyplot as plt
newchecker.runInfos['unmatchedEvents'].plot()

In [ ]:
unmatchedRun = newchecker.runInfos.groupby('run')
totalUnmatched = unmatchedRun['unmatchedEvents'].sum()
total = unmatchedRun['totalEvents'].sum()

frac = totalUnmatched / total


In [ ]:
plt.scatter(totalUnmatched.index, totalUnmatched.values, marker='o')
plt.yscale('log')
plt.title('Number of Unmatched Trigger Events')
plt.ylabel("Number of Events")
plt.xlabel("Run Number")

In [ ]:
plt.scatter(frac.index, frac.values, marker='o')
plt.yscale('log')
plt.title('Fraction of Unmatched Trigger Events')
plt.ylabel("Fraction of Total Events")
plt.xlabel("Run Number")